In [1]:
import tifffile.tifffile as tiff

image = tiff.imread('../../../../../Downloads/ts11.mrc_26.80Apx_semantic.tif')

In [2]:
from tardis_pytorch.dist_pytorch.utils.build_point_cloud import BuildPointCloud

post_processes = BuildPointCloud()
pc_hd, pc_ld = post_processes.build_point_cloud(
                image=image, EDT=True, down_sampling=5,
            )

In [3]:
from tardis_pytorch.utils.visualize_pc import VisualizePointCloud
VisualizePointCloud(pc_ld, False)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [4]:
from tardis_pytorch.dist_pytorch.datasets.patches import PatchDataSet
patch_pc = PatchDataSet(
                max_number_of_points=1000, graph=False
            )
coords_df, _, output_idx, _ = patch_pc.patched_dataset(
                    coord=pc_ld
                )

In [5]:
from tardis_pytorch.utils.predictor import Predictor

graphs = []
predict_dist = Predictor(
                network="dist",
                subtype="triang",
                model_type="microtubules",
                device='mps',
            )

for id_dist, coord in enumerate(coords_df):
    graph = predict_dist.predict(x=coord[None, :])
    graphs.append(graph)

Searching for weight file for dist_triang...


In [23]:
pc_ld = pc_ld * 26.8
pc_ld[:, 0] = pc_ld[:, 0]
pc_ld[:, 1] = pc_ld[:, 1]
pc_ld[:, 2] = pc_ld[:, 2]

In [26]:
from tardis_pytorch.dist_pytorch.utils.segment_point_cloud import PropGreedyGraphCut


GraphToSegment = PropGreedyGraphCut(
                    threshold=0.5, smooth=True
                )

segments = GraphToSegment.patch_to_segment(
                    graph=graphs, coord=pc_ld, idx=output_idx, prune=10
                )

In [24]:
from tardis_pytorch.utils.visualize_pc import VisualizeFilaments
import numpy as np
VisualizeFilaments(segments)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [27]:
from tardis_pytorch.utils.spline_metric import FilterSpatialGraph
filter_splines = FilterSpatialGraph(
                    connect_seg_if_closer_then=2500,
                    cylinder_radius=250,
                    filter_short_segments=500,
                )

segments_filter = filter_splines(segments=segments)

VisualizeFilaments(segments_filter)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
